In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
# Yes, I realize the error, but too many patterns made...
BrickRatio = 215 / 65  # [dimensionless]

# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm

In [3]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 12  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]

# Code:

In [4]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    flip,  # Flip the matrix, reduces travel time.
)

Lines parallel to the X-axis.
Separates rows of bricks.

In [6]:
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),# Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), # Even rows.
)
vertical_brick_lines_tuple

(array([ 24.53205128,  49.06410256,  73.59615385,  98.12820513,
        122.66025641, 147.19230769, 171.72435897]),
 array([ 12.26602564,  36.79807692,  61.33012821,  85.86217949,
        110.39423077, 134.92628205, 159.45833333]))

Lines parallel to the Y-axis.

In [9]:
horizontal_brick_lines = np.linspace(0, BlockHeight, N_BrickRows+1, endpoint=True)
horizontal_brick_lines

array([ 0.        ,  7.41666667, 14.83333333, 22.25      , 29.66666667,
       37.08333333, 44.5       , 51.91666667, 59.33333333, 66.75      ,
       74.16666667, 81.58333333, 89.        ])

In [10]:
BlockBrick=GCode.Program()
BlockBrick.lines=list()

BlockBrick2=GCode.Program()
BlockBrick2.lines=list()

for idx in range(1, len(horizontal_brick_lines)-1):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    line_ = GCode.Line(
        points=row_line_points,
    ) 
    BlockBrick.lines.append(line_)
    
    
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(
        points=row_line_points,
    )
    BlockBrick2.lines.append(line_)

    
BlockBrick.lines
BlockBrick.dist

3935.3500874434603

In [11]:
len(BlockBrick.lines)

11

In [12]:
BlockBrick2.dist

2416.0212033333337

In [13]:
BlockBrick.dist-BlockBrick2.dist

1519.3288841101266

In [14]:
assert np.isclose(BlockBrick.dist, 3935.3500874434603), "Something's Broken."
assert np.isclose(BlockBrick2.dist, 2416.0212033333337), "Something's Broken."

In [15]:
for idx in range(1, 10, 2):
    print(idx)

1
3
5
7
9


In [34]:
horizontal_brick_lines.

array([ 0.        ,  7.41666667, 14.83333333, 22.25      , 29.66666667,
       37.08333333, 44.5       , 51.91666667, 59.33333333, 66.75      ,
       74.16666667, 81.58333333])

In [16]:
for idx in range(1, len(horizontal_brick_lines)):
    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]
    
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]

In [ ]:

    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)

In [9]:
# Not drawing the last row of vertical lines.
idx+=1
vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]

start_point_y = horizontal_brick_lines[idx-1]
end_point_y = BlockHeight
        
for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
    transform = transform_tuple[np.mod(idx2, 2)]
    column_line_points = (
        np.array(
            [
                [vertical_brick_line, start_point_y],
                [vertical_brick_line, end_point_y],
            ]
        ),
    )
    column_line_points = np.matmul(transform, column_line_points)
    line_ = GCode.Line(column_line_points)
    BlockBrick.lines.append(line_)

In [10]:
BlockBrick.lines

[Line<len=178.15445mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=37.53805mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=86.18191mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=135.12997mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=178.0mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=51.25733mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=99.243mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=147.93784mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=179.38523mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=43.00187mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=88.69823mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=136.74854mm, feed=300, power=150>,

In [11]:
BlockBrick.lines[1]

Line<len=7.41667mm, feed=300, power=150>

In [12]:
l = BlockBrick.lines[1]

In [13]:
l.x_0

12.26602564102564

In [14]:
BlockBrick.time

3313.6085462389365

In [15]:
BlockBrick.dist

8175.817932477876

In [16]:
BlockBrick.generate_gcode()

In [17]:
BlockBrick

Program<lines=95, dist=8175.82mm, t=3313.61s>

In [18]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()

(4, ['', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]"])

In [19]:
cnc.status

'<Alarm|MPos:0.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [20]:
cnc.home()

14

In [21]:
cnc.status

'<Idle|MPos:-259.000,-174.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [22]:
BlockBrick.machine=cnc

In [ ]:
BlockBrick.run()